# Optimization of Flywheel Parameters

In [1]:
import warnings
warnings.simplefilter('ignore')

# declare units
m = units.length.meter
cm = units.length.centimeter
mm = units.length.millimeter
kg = units.mass.kilogram
g = units.mass.gram
s = units.time.second
Hz = units.frequency.hertz

# Some physical constants of our setup:
motor = {
    'max_rpm': 12000 * Hz,
    'reverse': True,
    'rotor_inertia': 0.69 * g * cm^2,
    'shaft_radius': 1.5 * mm,
    'mass': 6.9 * g
}

wheel = {
    'density': 7.45 * g / cm^3,
    'r_f': 0 * mm,
    'R_i': motor['shaft_radius'],
    'R_s': 0 * mm,
    'R_im': 10.5 * mm,
    'R_om': 12.5 * mm,
    'H_d': 2 * mm,
    'H_m': 10 * mm,
    'H_s': 0 * mm,
    'max_mass': 15 * g
}

%display latex

# declare variables
r_f, R_i, R_s, R_im, R_om, H_d, H_m, H_s, rho = var( 'r_f R_i R_s R_im R_om H_d H_m H_s rho' )
# some assumptions to make it easier for the symbolic optimizer
assume( R_i > 0 )
assume( R_s > R_i )
assume( R_im > R_s )
assume( R_om > R_im )
assume( r_f > 0 )
assume( H_d > 0 )
assume( H_d < H_s )
assume( H_d < H_m )
assume( R_im - r_f > 0 )

## Mass Formula:

In [2]:
# these are just the masses of the cylindrical components
sheath_mass = rho * pi * H_s * ( R_s^2 - R_i^2 )
disc_mass = rho * pi * H_d * ( R_im^2 - R_s^2 )
mass_mass = rho * pi * H_m * ( R_om^2 - R_im^2 )
cylindrical_masses = sheath_mass + disc_mass + mass_mass
# circular fillets
fillet_inner = (r_f - sqrt(r_f^2 - (x-(R_s+r_f))^2))
fillet_outer = (r_f - sqrt(r_f^2 - (x-(R_im-r_f))^2))
# masses of the fillets as a revolved integral
fillet_inner_mass = 2*pi*rho*integral( fillet_inner*x, (x, R_s, R_s+r_f) )
fillet_outer_mass = 2*pi*rho*integral( fillet_outer*x, (x, R_im-r_f, R_im) )
# grand unified mass equation:
total_mass = (
    cylindrical_masses +
    fillet_inner_mass +
    fillet_outer_mass + 
    motor['mass'] ).full_simplify()

## Inertia Formula:

In [3]:
# another shortcut for moments of inertia on the cylindrical component s:
sheath_inertia = sheath_mass / 2 * (R_i^2 + R_s^2)
disc_inertia = disc_mass / 2 * (R_s^2 + R_im^2)
mass_inertia = mass_mass / 2 * (R_im^2 + R_om^2)
# total moment of inertia for all 3 cylindrical components
cylindrical_inertias = sheath_inertia + disc_inertia + mass_inertia
# moments of inertia for the fillets
fillet_inner_inertia = 2*pi*rho*integral( fillet_inner*x^2, (x, R_s, R_s+r_f) )
fillet_outer_inertia = 2*pi*rho*integral( fillet_outer*x^2, (x, R_im-r_f, R_im) )
# grand unified inertia equation:
total_inertia = (
    cylindrical_inertias +
    fillet_inner_inertia +
    fillet_outer_inertia +
    motor['rotor_inertia'] ).full_simplify()

## Wheel mass for given parameters:

In [4]:
(total_mass(
    R_i=wheel['R_i'],
    R_s=wheel['R_s'],
    R_im=wheel['R_im'],
    R_om=wheel['R_om'],
    r_f=wheel['r_f'],
    H_d=wheel['H_d'],
    H_s=wheel['H_s'],
    H_m=wheel['H_m'],
    rho=wheel['density'] ).convert( g ) / g).n(30) * g

22.827011*gram

## Wheel inertia for given parameters:

In [7]:
inertia = ( total_inertia(
    R_i=wheel['R_i'],
    R_s=wheel['R_s'],
    R_im=wheel['R_im'],
    R_om=wheel['R_om'],
    r_f=wheel['r_f'],
    H_d=wheel['H_d'],
    H_s=wheel['H_s'],
    H_m=wheel['H_m'],
    rho=wheel['density'] ).convert( g * mm^2 ) / ( g * mm^2 ) ).n( 30 ) * g * mm^2

inertia

1788.0888*gram*millimeter^2

## Total stored momentum per wheel:

In [6]:
( inertia * motor['max_rpm'] * ( 2 if motor['reverse'] else 1 ) ).convert( g * m^2 / s )

42.914132*(gram*meter^2/second)